In [13]:
import re
import io
import requests
from bs4 import BeautifulSoup

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [25]:
url = 'https://www.al-akhbar.com/Editions'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

body = soup.findChildren()

In [29]:
body

[<html class="no-js" lang="en-US"> <!--<![endif]-->
 <head>
 <title>Attention Required! | Cloudflare</title>
 <meta charset="utf-8"/>
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
 <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
 <meta content="noindex, nofollow" name="robots"/>
 <meta content="width=device-width,initial-scale=1" name="viewport"/>
 <link href="/cdn-cgi/styles/cf.errors.css" id="cf_styles-css" rel="stylesheet"/>
 <!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->
 <style>body{margin:0;padding:0}</style>
 <!--[if gte IE 10]><!-->
 <script>
   if (!navigator.cookieEnabled) {
     window.addEventListener('DOMContentLoaded', function () {
       var cookieEl = document.getElementById('cookie-alert');
       cookieEl.style.display = 'block';
     })
   }
 </script>
 <!--<![endif]-->
 </head>
 <body>
 <div id="cf-wrapper">
 <div class="cf-alert cf-alert-error cf-cookie-error" data

In [18]:
pdf_links soup.find_all('a', href=lambda x: x and x.startswith('/Editions'))

[]

In [2]:
def save_article_to_drive(article_text, file_name):
  try:
      # authenticate with Google Drive
      credentials = Credentials.from_authorized_user_info()
      service = build('drive', 'v3', credentials=credentials)

      # create the file on Google Drive
      file_metadata = {'name': file_name, 'mimeType': 'text/plain'}
      media = MediaIoBaseUpload(io.StringIO(article_text), mimetype='text/plain',chunksize=1024*1024, resumable=True)
      file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
      print(f'File ID: {file.get("id")}')
  except HttpError as error:
      print(f'An error occurred: {error}')
      file = None
  return file

In [9]:
def extract_text_from_pdf(pdf_content):
  resource_manager = PDFResourceManager()
  fake_file_handle = io.StringIO()
  converter = TextConverter(resource_manager, fake_file_handle)
  page_interpreter = PDFPageInterpreter(resource_manager, converter)

  with io.BytesIO(pdf_content) as fh:
    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
      page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

  # close open handles
  converter.close()
  fake_file_handle.close()

  return text

In [16]:
# scrape the website for PDF links



# # download each PDF and save it to Google Drive
# text_ls = [] 
# for link in pdf_links:
#   # download the PDF
#   pdf_url = link['href']
#   pdf_response = requests.get(pdf_url)
#   pdf_content = pdf_response.content

#   # extract the text from the PDF
#   text = extract_text_from_pdf(pdf_content)
#   text_ls.append(text)
print(soup)

<!DOCTYPE html>

<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Attention Required! | Cloudflare</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/cf.errors.css" id="cf_styles-css" rel="stylesheet"/>
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->
<style>body{margin:0;padding:0}</style>
<!--[if gte IE 10]><!-->
<script>
  if (!navigator.cookieEnabled) {
    window.addEventListener('DOMContentLoaded', function

In [14]:
text_ls

[]

In [ ]:
# search the text for keywords
keywords = ['keyword1', 'keyword2', 'keyword3']
for keyword in keywords:
  if re.search(keyword, text, re.IGNORECASE):
    # keyword found, save the article
    file_name = f'{keyword}_article.txt'
    save_article_to_drive(text, file_name)
    break